# Homework Batch I: Matrix Multiplication
## Davide Basso - SM3500450

In [5]:
from __future__ import annotations
from numbers import Number
from typing import List, Tuple
from random import random, seed
from sys import stdout
from timeit import timeit
import math
from matrix import *

1) implement the strassen matrix mult function to multiply two $2^n \times 2^n$ matrices by using the Strassen's algorithm;

In [6]:
def strassen_matrix_mult(A: Matrix, B: Matrix) -> Matrix:
    assert A.num_of_cols == B.num_of_rows , "Incompatible matrices"

    # Base case
    if max(A.num_of_rows, B.num_of_cols, A.num_of_cols) < 32:
        return gauss_matrix_mult(A, B)
    
    # Recursive step
    A11, A12, A21, A22 = get_matrix_quadrants(A)
    B11, B12, B21, B22 = get_matrix_quadrants(B)

    # First batch of sums has cost Theta(n^2)
    S1 = B12 - B22
    S2 = A11 + A12
    S3 = A21 + A22
    S4 = B21 - B11
    S5 = A11 + A22
    S6 = B11 + B22 
    S7 = A12 - A22 
    S8 = B21 + B22
    S9 = A11 - A21
    S10 = B11 + B12 

    # Recursive calls
    P1 = strassen_matrix_mult(A11, S1)
    P2 = strassen_matrix_mult(S2, B22)
    P3 = strassen_matrix_mult(S3, B11)
    P4 = strassen_matrix_mult(A22, S4)
    P5 = strassen_matrix_mult(S5, S6)
    P6 = strassen_matrix_mult(S7, S8)
    P7 = strassen_matrix_mult(S9, S10)

    # Second batch of sums has cost Theta(n^2)
    C11 = P5 + P4 - P2 + P6
    C12 = P1 + P2
    C21 = P3 + P4
    C22 = P5 + P1 - P3 - P7

    # Build resulting matrix
    result_matrix = Matrix([[0 for x in range(B.num_of_cols)] for y in range(A.num_of_rows)],
                            clone_matrix=False)
    
    # Copying Cij into the resulting matrix
    result_matrix.assign_submatrix(0, 0, C11)
    result_matrix.assign_submatrix(0, result_matrix.num_of_cols//2, C12)
    result_matrix.assign_submatrix(result_matrix.num_of_rows//2, 0, C21)
    result_matrix.assign_submatrix(result_matrix.num_of_rows//2, result_matrix.num_of_cols//2, C22)
    
    return result_matrix


2) generalize strassen matrix mult to deal with any kind of matrix pair that can be multiplied (possibly also non-square matrices) and prove that the asymptotic complexity does not change;

In [7]:
def next_power_of_two(n):
    return int(math.pow(2, math.ceil(math.log(n)/math.log(2))))

def is_power_of_two(n):
    return ((n & (n-1) == 0) and n != 0)

def is_even(n):
    return (n%2 == 0)

In [8]:
def generalized_pow_strassen_matrix_mult(A: Matrix, B: Matrix) -> Matrix:
    assert A.num_of_cols == B.num_of_rows , "Incompatible matrices"

    # find which dimension is the largest

    biggest = max(A.num_of_rows, A.num_of_cols, B.num_of_cols)

    # padding of matrices if needed
    # basically I'm taking into account the following cases:
    ####### non-square matrix
    ####### non power of 2 number of columns
    ####### non power of 2 number of rows
    if ((A.num_of_cols != A.num_of_rows) or (not(is_power_of_two(A.num_of_cols))) or (not(is_power_of_two(A.num_of_rows)))):
        A_padded = Matrix([[0 for y in range(next_power_of_two(biggest))] 
                            for x in range(next_power_of_two(biggest))], clone_matrix=False)
        A_padded.assign_submatrix(0,0,A)
    else:
        A_padded = A
    
    if ((B.num_of_cols != B.num_of_rows) or (not(is_power_of_two(B.num_of_cols))) or (not(is_power_of_two(B.num_of_rows)))):

        B_padded = Matrix([[0 for y in range(next_power_of_two(biggest))] 
                            for x in range(next_power_of_two(biggest))], clone_matrix=False)
        B_padded.assign_submatrix(0,0,B)
    else:
        B_padded = B
    
    return strassen_matrix_mult(A_padded, B_padded).submatrix(0, A.num_of_rows, 0, B.num_of_cols)
    

Padding with zeros the matrix given as input if this one is not square or has as number of rows or column a number which is not equal to $2^n$ makes us able to recall and use the trivial Strassen algorithm implementation. The resulting matrix will be a $N \times N$ matrix where $N$ will be the nearest power of two of the biggest size among rows and columns.

Another approach consists in adding just a row or a column if the total amount is equal to an odd number, this could also lead to a smaller overhead w.r.t previous method since we don't have to allocate such bigger matrix than the original ones. Here below the implementation:


In [9]:
def generalized_even_strassen_matrix_mult(A: Matrix, B: Matrix) -> Matrix:
    assert A.num_of_cols == B.num_of_rows , "Incompatible matrices"

    if(A.num_of_cols==A.num_of_rows and B.num_of_cols==B.num_of_rows) and (is_power_of_two(A.num_of_rows)):
        return strassen_matrix_mult(A,B)

    if max(A.num_of_rows, B.num_of_cols, A.num_of_cols) < 32:
        return gauss_matrix_mult(A, B)
    
    # next even padding
    A_even_row = A.num_of_rows + A.num_of_rows%2
    A_even_col = A.num_of_cols + A.num_of_cols%2
    B_even_row = B.num_of_rows + B.num_of_rows%2
    B_even_col = B.num_of_cols + B.num_of_cols%2
    
    A_padded = Matrix([[0 for y in range(A_even_col)] for x in range(A_even_row)], clone_matrix=False)
    A_padded.assign_submatrix(0,0,A)
    B_padded = Matrix([[0 for y in range(B_even_col)] for x in range(B_even_row)], clone_matrix=False)
    B_padded.assign_submatrix(0,0,B)
    
    # Recursive step
    A11, A12, A21, A22 = get_matrix_quadrants(A_padded)
    B11, B12, B21, B22 = get_matrix_quadrants(B_padded)

    # First batch of sums has cost Theta(n^2)
    S1 = B12 - B22
    S2 = A11 + A12
    S3 = A21 + A22
    S4 = B21 - B11
    S5 = A11 + A22
    S6 = B11 + B22 
    S7 = A12 - A22 
    S8 = B21 + B22
    S9 = A11 - A21
    S10 = B11 + B12 

    # Recursive calls
    P1 = generalized_even_strassen_matrix_mult(A11, S1)
    P2 = generalized_even_strassen_matrix_mult(S2, B22)
    P3 = generalized_even_strassen_matrix_mult(S3, B11)
    P4 = generalized_even_strassen_matrix_mult(A22, S4)
    P5 = generalized_even_strassen_matrix_mult(S5, S6)
    P6 = generalized_even_strassen_matrix_mult(S7, S8)
    P7 = generalized_even_strassen_matrix_mult(S9, S10)

    # Second batch of sums has cost Theta(n^2)
    C11 = P5 + P4 - P2 + P6
    C12 = P1 + P2
    C21 = P3 + P4
    C22 = P5 + P1 - P3 - P7

    # Build resulting matrix
    result_matrix = Matrix([[0 for y in range(B_even_col)] for x in range(A_even_row)],
                            clone_matrix=False)
    
    # Copying Cij into the resulting matrix
    result_matrix.assign_submatrix(0, 0, C11)
    result_matrix.assign_submatrix(0, result_matrix.num_of_cols//2, C12)
    result_matrix.assign_submatrix(result_matrix.num_of_rows//2, 0, C21)
    result_matrix.assign_submatrix(result_matrix.num_of_rows//2, result_matrix.num_of_cols//2, C22)
    
    return result_matrix.submatrix(0,A.num_of_rows,0,B.num_of_cols)


Checking if the results are compatible by taking the difference between the two resulting matrices, using both methods implemented before:

In [11]:
n=[59,31,10,38,91,234,387]
m=[37,61,42,35,73,212,315]

for i,j in zip(n,m):
    A = Matrix([[math.floor(random()*10) for k in range(j)]for l in range(i)])
    B = Matrix([[math.floor(random()*10) for k in range(i)]for l in range(j)])

    C = gauss_matrix_mult(A,B)
    D = generalized_even_strassen_matrix_mult(A,B)
    E = generalized_pow_strassen_matrix_mult(A,B)
    diff_even = C-D
    diff_pow = C-E
    sum1 = 0
    sum2 = 0

    for z in range(i):
        for y in range(i):
            sum1 = sum1 + diff_even[z][y]
            sum2 = sum2 + diff_pow[z][y]

    print(f'The difference between the two matrices using next power of 2 method is = {sum1}')
    print(f'The difference between the two matrices using next even number method is = {sum2}')

The difference between the two matrices using next power of 2 method is = 0
The difference between the two matrices using next even number method is = 0
The difference between the two matrices using next power of 2 method is = 0
The difference between the two matrices using next even number method is = 0
The difference between the two matrices using next power of 2 method is = 0
The difference between the two matrices using next even number method is = 0
The difference between the two matrices using next power of 2 method is = 0
The difference between the two matrices using next even number method is = 0
The difference between the two matrices using next power of 2 method is = 0
The difference between the two matrices using next even number method is = 0
The difference between the two matrices using next power of 2 method is = 0
The difference between the two matrices using next even number method is = 0
The difference between the two matrices using next power of 2 method is = 0
The di

It's important to notice that for both methods, with padding we are not affecting asymptotic complexity since in the worst case we will enlarge the number of columns or rows up to $N$ which is $<2n$ (for the first method e.g $n=1025$ will become $N=2048$). So knowing that time complexity for Strassen algorithm is $O(n^{log_27})$ we can prove that this actually is left unchanged:
$$ O(N^{log_27}) < O(2n^{log_27}) = O(7n^{log_27}) \in O(n^{log_27}) $$


Finally we can make a comparison between timings using the usual Gauss method, the padding to next even number and to next power of 2:

In [12]:
seed(0)

for i in range(7):
    size = 3**i
    stdout.write(f'{size}') 
    A = Matrix([[random() for x in range(size)] for y in range(size)])
    B = Matrix([[random() for x in range(size)] for y in range(size)])

    for funct in ['gauss_matrix_mult','generalized_even_strassen_matrix_mult','generalized_pow_strassen_matrix_mult']:
        T = timeit(f'{funct}(A,B)', globals=locals(), number=1)
        stdout.write('\t{:.3f}'.format(T))
        stdout.flush()
    stdout.write('\n')

1	0.000	0.000	0.000
3	0.000	0.000	0.000
9	0.000	0.000	0.003
27	0.007	0.009	0.017
81	0.231	0.227	0.804
243	5.182	4.493	5.359
729	166.004	110.730	288.919


Results show that the latter method is strongly affected by the instantiation of a much larger matrix w.r.t the original one, so even if the complexity is the same it is not convenient to use that. On the other hand we can observe that the generalized method using padding to the next even number is very efficient and can lead to strong benefits.

3) improve the implementation of the function by reducing the number of auxiliary matrices and test the effects on the execution time;

The idea I followed in order to reduce the number of auxiliary matrices was to store within the original matrices $A$ and $B$ the partial sums $S_i$ (for $i = 1,...,10$), using also the final matrix C which is initialized as empty. 
I decided to leave the $P_i$ matrices since it would be too much inconvenient for what concerns computational timings to recompute each matrix $P_i$ for each one of the final quadrant $C_{ij}$. Moreover, regarding the final quadrants, we can avoid to compute the summations and differences while initializing 4 new matrices and pass them directly as function arguments.
I would like to point out that this solution actualy loses the memory advantages by using the function submatrix in the definition of all $P_i$ matrices since submatrix() creates a copy.


In [13]:
def opt_strassen_matrix_mult(A: Matrix, B: Matrix) -> Matrix:

    # Base case
    if max(A.num_of_rows, B.num_of_cols, A.num_of_cols) < 32:
        return gauss_matrix_mult(A, B)
    
    # Recursive step
    A11, A12, A21, A22 = get_matrix_quadrants(A)
    B11, B12, B21, B22 = get_matrix_quadrants(B)

    # Build resulting matrix
    result_matrix = Matrix([[0 for x in range(B.num_of_cols)] for y in range(A.num_of_rows)],
                            clone_matrix=False)

    # In order to reuse same memory space we allocate 
    # the matrices within the original ones
    B.assign_submatrix(0,0,B12 - B22)
    A.assign_submatrix(0,0,A11 + A12)
    A.assign_submatrix(0,A.num_of_cols//2,A21 + A22)
    B.assign_submatrix(0,B.num_of_cols//2,B21 - B11)
    A.assign_submatrix(A.num_of_rows//2,0,A11 + A22)
    B.assign_submatrix(B.num_of_rows//2,0,B11 + B22)
    A.assign_submatrix(A.num_of_rows//2,A.num_of_cols//2,A12 - A22) 
    B.assign_submatrix(B.num_of_rows//2,B.num_of_cols//2,B21 + B22)
    result_matrix.assign_submatrix(0,0,A11 - A21)
    result_matrix.assign_submatrix(0,result_matrix.num_of_cols//2,B11 + B12) 

    # Recursive calls
    P1 = strassen_matrix_mult(A11, B.submatrix(0, B.num_of_rows//2, 0, B.num_of_cols//2))
    P2 = strassen_matrix_mult(A.submatrix(0, A.num_of_rows//2, 0, A.num_of_cols//2), B22)
    P3 = strassen_matrix_mult(A.submatrix(0, A.num_of_rows//2, A.num_of_cols//2, A.num_of_cols//2), B11)
    P4 = strassen_matrix_mult(A22, B.submatrix(0, B.num_of_rows//2, B.num_of_cols//2, B.num_of_cols//2))
    P5 = strassen_matrix_mult(A.submatrix(A.num_of_rows//2, A.num_of_rows//2, 0, A.num_of_cols//2), 
                              B.submatrix(B.num_of_rows//2, B.num_of_rows//2, 0, B.num_of_cols//2))
    P6 = strassen_matrix_mult(A.submatrix(A.num_of_rows//2, A.num_of_rows//2, A.num_of_cols//2, A.num_of_cols//2), 
                              B.submatrix(B.num_of_rows//2, B.num_of_rows//2, B.num_of_cols//2, B.num_of_cols//2))
    P7 = strassen_matrix_mult(result_matrix.submatrix(0, result_matrix.num_of_rows//2, 0, result_matrix.num_of_cols//2), 
                              result_matrix.submatrix(0, result_matrix.num_of_rows//2, result_matrix.num_of_cols//2,                                            result_matrix.num_of_cols//2))
    
    # Copying Cij into the resulting matrix
    result_matrix.assign_submatrix(0, 0, P5 + P4 - P2 + P6)
    result_matrix.assign_submatrix(0, result_matrix.num_of_cols//2, P1 + P2)
    result_matrix.assign_submatrix(result_matrix.num_of_rows//2, 0, P3 + P4)
    result_matrix.assign_submatrix(result_matrix.num_of_rows//2, result_matrix.num_of_cols//2, P5 + P1 - P3 - P7)
    
    return result_matrix


In the code below I tried to implement the same thing as above but without putting in the already existent matrices the result of the summations.


In [14]:
def opt2_strassen_matrix_mult(A: Matrix, B: Matrix) -> Matrix:

    # Base case
    if max(A.num_of_rows, B.num_of_cols, A.num_of_cols) < 32:
        return gauss_matrix_mult(A, B)
    
    # Recursive step
    A11, A12, A21, A22 = get_matrix_quadrants(A)
    B11, B12, B21, B22 = get_matrix_quadrants(B)

    # Recursive calls
    P1 = strassen_matrix_mult(A11, B12 - B22)
    P2 = strassen_matrix_mult(A11 + A12, B22)
    P3 = strassen_matrix_mult(A21 + A22, B11)
    P4 = strassen_matrix_mult(A22, B21 - B11)
    P5 = strassen_matrix_mult(A11 + A22, B11 + B22)
    P6 = strassen_matrix_mult(A12 - A22 , B21 + B22)
    P7 = strassen_matrix_mult(A11 - A21, B11 + B12)

    # Build resulting matrix
    result_matrix = Matrix([[0 for x in range(B.num_of_cols)] for y in range(A.num_of_rows)],
                            clone_matrix=False)
    
    # Copying Cij into the resulting matrix
    result_matrix.assign_submatrix(0, 0, P5 + P4 - P2 + P6)
    result_matrix.assign_submatrix(0, result_matrix.num_of_cols//2, P1 + P2)
    result_matrix.assign_submatrix(result_matrix.num_of_rows//2, 0, P3 + P4)
    result_matrix.assign_submatrix(result_matrix.num_of_rows//2, result_matrix.num_of_cols//2, P5 + P1 - P3 - P7)
    
    return result_matrix


Here the testings in order to check the effects on execution time:

In [15]:
seed(0)

for i in range(11):
    size = 2**i
    stdout.write(f'{size}') 
    A = Matrix([[random() for x in range(size)] for y in range(size)])
    B = Matrix([[random() for x in range(size)] for y in range(size)])

    for funct in ['gauss_matrix_mult','strassen_matrix_mult','opt_strassen_matrix_mult','opt2_strassen_matrix_mult']:
        T = timeit(f'{funct}(A,B)', globals=locals(), number=1)
        stdout.write('\t{:.3f}'.format(T))
        stdout.flush()
    stdout.write('\n')

1	0.000	0.000	0.000	0.000
2	0.000	0.000	0.000	0.000
4	0.000	0.000	0.000	0.000
8	0.000	0.001	0.001	0.001
16	0.003	0.003	0.004	0.004
32	0.025	0.025	0.022	0.018
64	0.152	0.119	0.106	0.112
128	0.766	0.731	0.779	0.746
256	5.949	5.264	5.224	5.250
512	47.174	36.379	36.206	36.630
1024	385.911	256.303	255.604	255.094


We can se that actually the first optimization method is the best among all the methods, but only slightly if we consider the Strassen approach.

4) answer to the following question: how much is the minimum auxiliary
space required to evaluate the Strassen's algorithm? Motivate the answer.

The minimum auxiliary space required to evaluate Strassen's algorithm without any kind of modifications is:

$\begin{aligned}
M(n) &= 7M(\frac{n}{2})+ \Theta(n^2), \space n>1 \\
M(n) &= 1, \space n = 1
\end{aligned}$

This is because at each iteration we need to allocate memory for the 10 partial sums (that follows $\Theta(n^2)$ complexity) and for the 7 matrices in which we store the partial multiplications.
As we can notice the formulation is the same as the one for arithmetic complexity of Strassen's algorithm, so the final result is that the minimum axiliary space required is $O(n^{log_27})$.

**Note**: The previous result is valid also for the optimized algorithm since we don't eliminate $P_i$ matrices that take advantage of the recursion.